In [3]:
%matplotlib inline

In [94]:
# LSTM and CNN for sequence classification in the IMDB dataset
import numpy
import numpy as np
import pandas as pd
import os
import sys

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import optimizers
import xgboost
import features_extraction
import importlib
features_extraction = importlib.reload(features_extraction)
from xgboost import XGBClassifier
import itertools
from sklearn.model_selection import train_test_split,KFold
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report, log_loss
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import model_selection 
# fix random seed for reproducibility
numpy.random.seed(7)


In [95]:
import sys
root =  "/home/bugabuga/hack/git"
sys.path.insert(0, root)
# import os
# os.chdir(root)
# import get_decreasing_data
# df = get_decpwdreasing_data.get_decreasing_df(None)


In [96]:
# df.to_pickle(os.path.expanduser("~/hack/data/decreasing_data.p"))

In [97]:
df = pd.read_pickle(os.path.expanduser("~/hack/data/decreasing_data.p"))
df.head()
# df.iloc[0].posX

,posX,posY,posZ,velX,velY,velZ,class
1,"[834.907832288, 1107.20354278, 1381.77279678, ...","[3.5280955550699997, 1.8188204949900002, 0.524...","[8076.75484631, 8075.48815853, 8065.34459174, ...","[550.590397994, 549.8443559140001, 553.2501705...","[0.646220001584, -2.74706134934, -2.1050146181...","[-1.18552315801, -5.95741751005, -12.404433416...",14
2,"[0, 217.255563152, 436.462066491, 647.14079424...","[0, -2.3429267468400004, -2.3307745953400003, ...","[7804.59700371, 7796.36193989, 7774.98015624, ...","[438.284571945, 432.800214042, 428.233753084, ...","[-0.340086091219, 3.7640691841600002, 5.437413...","[-13.597956928099999, -19.737817660999998, -23...",21
3,"[3378.51282212, 3593.79640812, 3802.05955668, ...","[4.50162900675, 1.1567393960899999, -3.4179981...","[18663.7080762, 18658.6674275, 18649.3245972, ...","[418.823720032, 422.404850152, 420.037047028, ...","[-2.67738188199, -3.48999138141, 0.80100161980...","[-5.27823493328, -7.491722379489999, -7.588743...",14
6,"[0, 120.12500347299999, 240.19124697599997, 35...","[0, 2.5878185792, -2.49164922338, -1.351626179...","[1841.89589425, 1721.70532429, 1595.39894829, ...","[238.0167313, 236.39145691299998, 232.23905293...","[3.62398367384, -1.9372451940900002, 1.8596656...","[-247.322457505, -251.370729304, -251.89407525...",3
8,"[0, 159.962582311, 321.827864744, 479.50818498...","[0, -1.5448295045700002, 2.3114140124, -3.8631...","[22755.0077447, 22730.722812, 22705.4872449, 2...","[322.90754832700003, 320.451017311, 325.864514...","[-2.11610096781, 2.9447404028, -1.62162975159,...","[-49.276934022, -49.8676433696, -55.0967237221...",16


In [98]:
# def pad_sampled_route(x):
#     arr = np.array(x[:-1].tolist(), dtype=np.float64)
#     padded = sequence.pad_sequences(arr, maxlen=30, padding='post', dtype='float64')
#     return padded.T
N = 1000
ts = 6
print(df.shape)
df_dropped = df[df.apply(lambda x: len(x["posX"])>ts, axis=1)].iloc[:N,:]
print(df_dropped.shape)

# df_rows = [df_features.iloc[i] for i in range(df_features.shape[0])]
# X = padded_data = np.array(list(map(pad_sampled_route, df_rows)))
# y = np.array(df_features['class'].tolist())


(17091, 7)
(1000, 7)


In [51]:
df_features = features_extraction.create_features(df_dropped, 3)


In [52]:
X = df_features.values

In [53]:
X.shape

(1000, 69)

In [101]:
X = df_features.values
X = preprocessing.StandardScaler().fit_transform(X)

y = df_dropped['class'].values

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=0)

In [106]:

clf = gbc = xgboost.XGBClassifier(objective='multi:softprob',
                    learning_rate=0.2,
                    subsample=0.7,
                    colsample_bytree=0.9,
                    colsample_bylevel=0.7,
                    max_depth=2,
                    nthread=4,
                    n_estimators=300,
                    seed=1234)
gbc.fit(X_train,y_train)


XGBClassifier(base_score=0.5, colsample_bylevel=0.7, colsample_bytree=0.9,
       gamma=0, learning_rate=0.2, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=300, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1234, silent=True, subsample=0.7)

In [107]:
np.mean(clf.predict(X_train) == y_train), np.mean(clf.predict(X_test) == y_test)

(1.0, 0.28999999999999998)

123123

In [55]:
def to_one_hot(labels, labels_count):
    result = np.zeros((labels.shape[0], labels_count), dtype=np.int32)
    for i in range(labels.shape[0]):
        result[i, labels[i]] = 1
    return result


In [87]:
pd.Series(y).value_counts()

3     74
7     65
8     63
1     60
10    60
6     58
4     52
5     51
2     42
12    40
21    40
9     40
24    37
13    35
11    32
14    32
16    28
20    28
18    25
19    25
22    25
15    24
17    24
23    20
25    20
dtype: int64

### Dense

In [99]:
n_classes = 26
dropout = 0.5
y_test_hot, y_train_hot = to_one_hot(y_test, n_classes), to_one_hot(y_train, n_classes)

model = Sequential()

n_features = X_train.shape[1]
hidden_lengths = [50, 50]

model.add(Dense(hidden_lengths[0], input_shape=(n_features,), activation='relu'))
model.add(Dropout(dropout))
for hidden_size in hidden_lengths[1:]:
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))
optimizer = optimizers.Adam(lr=0.01, decay=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 50)                3500      
_________________________________________________________________
dropout_21 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_22 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 26)                1326      
Total params: 7,376
Trainable params: 7,376
Non-trainable params: 0
_________________________________________________________________
None


In [100]:
for lr in [0.001, 0.0001, 0.00001]:
    optimizer = optimizers.Adam(lr=lr, decay=1e-4)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    for i in range(20):
        model.fit(X_train, y_train_hot, epochs=20, batch_size=100, verbose=0)
        # Final evaluation of the model
        scores_test = model.evaluate(X_test, y_test_hot, verbose=0)
        scores_train = model.evaluate(X_train, y_train_hot, verbose=0)
        print("train: {:.3f}, test: {:.3f}, lr: {}".format(scores_train[1], scores_test[1], lr))

train: 0.051, test: 0.050, lr: 0.001
train: 0.044, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.046, test: 0.010, lr: 0.001
train: 0.042, test: 0.010, lr: 0.001
train: 0.044, test: 0.010, lr: 0.001
train: 0.044, test: 0.010, lr: 0.0001
train: 0.042, test: 0.010, lr: 0.0001


KeyboardInterrupt: 